# 1. IMPORTAR LIBRERÍAS NECESARIAS


In [1]:
import pandas as pd
import numpy as np
from river import anomaly
from river import preprocessing
import pickle
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


# 2. CONFIGURACIÓN INICIAL Y PARÁMETROS


In [2]:
# Rutas de archivos
DATASET_PATH = r"C:\Users\User\Desktop\TESIS\Datasets\DataSetAgrupadoNoSuper3.csv"
MODELS_PATH = r"C:\Users\User\Desktop\TESIS\Modelos"

# Crear directorio de modelos si no existe
os.makedirs(MODELS_PATH, exist_ok=True)

# PARÁMETROS AJUSTABLES DEL MODELO ⚙️
N_TREES = 80  # 🔧 AJUSTABLE: 60-120 (más árboles = más precisión pero más lento)
TREE_HEIGHT = 10  # 🔧 AJUSTABLE: 8-15 (mayor altura = patrones más complejos)
MIN_RECORDS_WARMUP = 50  # Mínimo de registros para warm-up por país

# PARÁMETROS DE FEATURES ⚙️
PESO_MINUTOS_NORMAL = 0.4  # 🔧 AJUSTABLE: 0.2-0.6 (peso normal de minutos)
PESO_MINUTOS_EXTREMOS = 1.5  # 🔧 AJUSTABLE: 1.2-2.0 (peso cuando minutos son extremos)
UMBRAL_MINUTOS_EXTREMOS = 300  # 🔧 AJUSTABLE: 200-500 (minutos para considerar extremo)
PESO_DESTINOS = 1.2  # 🔧 AJUSTABLE: 1.0-1.5 (importancia de destinos)
PESO_SPRAY_RATIO = 1.5  # 🔧 AJUSTABLE: 1.2-2.0 (importancia del ratio spray)

# PARÁMETROS DE UMBRAL ⚙️
PERCENTIL_BASE = 99  # 🔧 AJUSTABLE: 95-99.5 (percentil para umbral)
AJUSTE_UMBRAL = 1.0  # 🔧 AJUSTABLE: 0.8-1.2 (multiplicador del umbral)

print(f"📁 Directorio de modelos: {MODELS_PATH}")
print(f"🤖 Modelo general con {N_TREES} árboles, altura {TREE_HEIGHT}")
print(f"⚙️ Parámetros configurables definidos")

📁 Directorio de modelos: C:\Users\User\Desktop\TESIS\Modelos
🤖 Modelo general con 80 árboles, altura 10
⚙️ Parámetros configurables definidos


# 3. CARGAR Y EXPLORAR EL DATASET

In [3]:
print("\n🔄 Cargando dataset...")
df = pd.read_csv(DATASET_PATH)

# Convertir fecha a datetime
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y', errors='coerce')

print(f"📋 Dataset cargado - Shape: {df.shape}")
print(f"📅 Rango de fechas: {df['FECHA'].min()} a {df['FECHA'].max()}")
print(f"🌍 Países únicos: {df['CODIGODEPAIS'].nunique()}")
print(f"📞 Líneas únicas: {df['LINEA'].nunique()}")

# Mostrar estadísticas generales
print(f"\n📊 Estadísticas generales:")
print(f"📞 Llamadas - Min: {df['N_LLAMADAS'].min()}, Max: {df['N_LLAMADAS'].max()}, Media: {df['N_LLAMADAS'].mean():.1f}")
print(f"⏱️ Minutos - Min: {df['N_MINUTOS'].min()}, Max: {df['N_MINUTOS'].max()}, Media: {df['N_MINUTOS'].mean():.1f}")
print(f"🎯 Destinos - Min: {df['N_DESTINOS'].min()}, Max: {df['N_DESTINOS'].max()}, Media: {df['N_DESTINOS'].mean():.1f}")



🔄 Cargando dataset...
📋 Dataset cargado - Shape: (794810, 6)
📅 Rango de fechas: 2025-03-01 00:00:00 a 2025-04-24 00:00:00
🌍 Países únicos: 188
📞 Líneas únicas: 458606

📊 Estadísticas generales:
📞 Llamadas - Min: 1, Max: 579, Media: 1.9
⏱️ Minutos - Min: 0.02, Max: 1019.8, Media: 5.8
🎯 Destinos - Min: 1, Max: 546, Media: 1.7


# 4. DIVISIÓN TEMPORAL PARA WARM-UP Y SCORING

In [4]:
df_sorted = df.sort_values('FECHA')
fechas_unicas = sorted(df['FECHA'].dt.to_period('M').unique())

if len(fechas_unicas) >= 2:
    primer_mes = fechas_unicas[0]
    segundo_mes = fechas_unicas[1]
    
    df_warmup = df[df['FECHA'].dt.to_period('M') == primer_mes].copy()
    df_scoring = df[df['FECHA'].dt.to_period('M') == segundo_mes].copy()
    
    print(f"\n📆 Período de warm-up: {primer_mes}")
    print(f"📆 Período de scoring: {segundo_mes}")
else:
    # División por mediana de fechas
    fecha_corte = df_sorted['FECHA'].quantile(0.5)
    df_warmup = df[df['FECHA'] <= fecha_corte].copy()
    df_scoring = df[df['FECHA'] > fecha_corte].copy()
    
    print(f"\n📆 División por fecha de corte: {fecha_corte}")

print(f"🔢 Registros warm-up: {len(df_warmup)}")
print(f"🔢 Registros scoring: {len(df_scoring)}")


📆 Período de warm-up: 2025-03
📆 Período de scoring: 2025-04
🔢 Registros warm-up: 450900
🔢 Registros scoring: 343910


# 5. ANÁLISIS DE CONTEXTO POR PAÍS (PARA NORMALIZACIÓN)


In [5]:
print(f"\n🌍 Analizando contexto por país...")

# Calcular estadísticas por país en período de warm-up
stats_por_pais = df_warmup.groupby('CODIGODEPAIS').agg({
    'N_LLAMADAS': ['count', 'mean', 'std', lambda x: x.quantile(0.90), lambda x: x.quantile(0.95)],
    'N_MINUTOS': ['mean', 'std', lambda x: x.quantile(0.90), lambda x: x.quantile(0.95)],
    'N_DESTINOS': ['mean', 'std', lambda x: x.quantile(0.90), lambda x: x.quantile(0.95)]
}).round(2)

stats_por_pais.columns = ['REGISTROS', 'LLAMADAS_MEAN', 'LLAMADAS_STD', 'LLAMADAS_P90', 'LLAMADAS_P95',
                         'MINUTOS_MEAN', 'MINUTOS_STD', 'MINUTOS_P90', 'MINUTOS_P95',
                         'DESTINOS_MEAN', 'DESTINOS_STD', 'DESTINOS_P90', 'DESTINOS_P95']

# Clasificar países por volumen de tráfico
stats_por_pais['CATEGORIA'] = pd.cut(stats_por_pais['REGISTROS'], 
                                   bins=[0, 50, 200, 1000, float('inf')],
                                   labels=['Muy_Bajo', 'Bajo', 'Medio', 'Alto'])

print(f"📊 Distribución de países por tráfico:")
print(stats_por_pais['CATEGORIA'].value_counts())

# Mostrar ejemplos por categoría
print(f"\n🔍 Ejemplos por categoría de tráfico:")
for categoria in ['Muy_Bajo', 'Bajo', 'Medio', 'Alto']:
    paises_cat = stats_por_pais[stats_por_pais['CATEGORIA'] == categoria]
    if len(paises_cat) > 0:
        print(f"\n{categoria} ({len(paises_cat)} países):")
        print(paises_cat[['REGISTROS', 'LLAMADAS_MEAN', 'DESTINOS_MEAN', 'MINUTOS_MEAN']].head(3))



🌍 Analizando contexto por país...
📊 Distribución de países por tráfico:
CATEGORIA
Muy_Bajo    129
Bajo         21
Medio        20
Alto         18
Name: count, dtype: int64

🔍 Ejemplos por categoría de tráfico:

Muy_Bajo (129 países):
              REGISTROS  LLAMADAS_MEAN  DESTINOS_MEAN  MINUTOS_MEAN
CODIGODEPAIS                                                       
27                   26           1.27           1.15          0.40
40                   24           1.38           1.00          0.40
62                   29           1.07           1.07          0.14

Bajo (21 países):
              REGISTROS  LLAMADAS_MEAN  DESTINOS_MEAN  MINUTOS_MEAN
CODIGODEPAIS                                                       
20                   81           1.22           1.01          0.18
30                   98           1.37           1.16          2.35
36                  120           1.23           1.02          0.10

Medio (20 países):
              REGISTROS  LLAMADAS_MEAN  DESTIN

# 6. FUNCIÓN PARA CREAR CARACTERÍSTICAS CONTEXTUALIZADAS

In [6]:
def crear_features_contextualizadas_mejorada(row, stats_pais_dict):
    """
    Crea características balanceadas que detecten minutos extremos y spray calling
    """
    pais = row['CODIGODEPAIS']
    llamadas = row['N_LLAMADAS']
    minutos = row['N_MINUTOS']
    destinos = row['N_DESTINOS']
    
    # Obtener contexto del país (si existe)
    if pais in stats_pais_dict:
        pais_stats = stats_pais_dict[pais]
        categoria = pais_stats['CATEGORIA']
        
        # Normalizar por el contexto del país
        llamadas_norm = min(llamadas / max(pais_stats['LLAMADAS_P95'], 1), 1.5)
        destinos_norm = min(destinos / max(pais_stats['DESTINOS_P95'], 1), 1.5)
        
        # MEJORA: Detección inteligente de minutos extremos
        minutos_p90 = pais_stats.get('MINUTOS_P90', pais_stats['MINUTOS_P95'] * 0.9)
        minutos_p95 = pais_stats['MINUTOS_P95']
        
        # Transformación adaptativa de minutos
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:  # 🔧 Minutos extremos
            minutos_norm = min(minutos / max(minutos_p90, 1), 3.0)  # Mayor rango para extremos
            peso_minutos = PESO_MINUTOS_EXTREMOS  # Peso alto para extremos
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(max(minutos_p90, 1)), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL  # Peso normal
            
    else:
        # País nuevo - SIEMPRE clasificar como 'Muy_Bajo'
        categoria = 'Muy_Bajo'
        llamadas_norm = min(llamadas / 10, 2.0)  # Más sensible para países nuevos
        destinos_norm = min(destinos / 5, 2.0)   # Más sensible para países nuevos
        
        # Para países nuevos, ser más sensible a minutos altos
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / 50, 3.0)  # Muy sensible a minutos extremos
            peso_minutos = PESO_MINUTOS_EXTREMOS * 1.2  # Peso extra para países nuevos
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(60), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
    
    # Features principales - REBALANCEADAS con detección de extremos
    features = {
        # 1. Valores normalizados con peso adaptativo
        'llamadas_norm': llamadas_norm * 0.8,
        'destinos_norm': destinos_norm * PESO_DESTINOS,  # 🔧 Ajustable
        'minutos_norm': minutos_norm * peso_minutos,     # 🔧 Peso adaptativo
        
        # 2. Ratios críticos para fraude
        'diversidad_destinos': min(destinos / max(llamadas, 1), 1.0),
        'spray_ratio': min(destinos / max(llamadas, 1) * PESO_SPRAY_RATIO, 1.0) if destinos >= 5 else 0,
        
        # 3. NUEVA: Detección específica de minutos extremos
        'minutos_extremos': 1.0 if minutos >= UMBRAL_MINUTOS_EXTREMOS else 0.0,
        'minutos_sospechosos': min((minutos - 200) / 300, 1.0) if minutos > 200 else 0.0,
        
        # 4. Patrones de fraude
        'patron_spray_fuerte': 1.0 if (destinos >= 10 and llamadas >= 20) else 0.0,
        'patron_spray_medio': 0.5 if (destinos >= 6 and llamadas >= 12) else 0.0,
        'alta_diversidad': min(destinos / 12, 1) if destinos >= 5 else 0,
        
        # 5. Indicadores de volumen anómalo
        'volumen_llamadas_alto': min((llamadas - 30) / 50, 1) if llamadas > 30 else 0,
        'volumen_destinos_alto': min((destinos - 10) / 20, 1) if destinos > 10 else 0,
        
        # 6. Características de comportamiento
        'llamadas_por_destino': min(llamadas / max(destinos, 1) / 5, 1),
        'eficiencia_destinos': min(destinos / max(llamadas * 0.5, 1), 1),
        
        # 7. MEJORA: Ajuste por contexto de país
        'factor_pais_bajo': 1.5 if categoria in ['Muy_Bajo', 'Bajo'] else 1.0,  # Más sensible
        'factor_pais_alto': 0.9 if categoria in ['Alto', 'Medio'] else 1.0      # Menos sensible
    }
    
    return features

# Convertir stats a diccionario para búsqueda rápida
stats_dict = {}
for pais, row in stats_por_pais.iterrows():
    stats_dict[pais] = row.to_dict()

print("🔧 Función de features mejorada definida (detección de minutos extremos)")


🔧 Función de features mejorada definida (detección de minutos extremos)


# 7. ENTRENAMIENTO DEL MODELO GENERAL


In [7]:
print(f"\n🤖 Entrenando modelo con detección de extremos...")

# Crear modelo con parámetros configurables
modelo_mejorado = anomaly.HalfSpaceTrees(
    n_trees=N_TREES,
    height=TREE_HEIGHT,
    seed=42
)

# Crear scaler
scaler_mejorado = preprocessing.StandardScaler()

# Procesar datos con features mejoradas
scores_mejorados = []
features_mejoradas = []

print("🔄 Procesando con features mejoradas...")
for contador, (idx, row) in enumerate(df_warmup.iterrows()):
    if contador % 2000 == 0:
        print(f"   Procesado: {contador}/{len(df_warmup)} registros")
    
    # Usar función mejorada
    features = crear_features_contextualizadas_mejorada(row, stats_dict)
    
    # Normalizar
    scaler_mejorado.learn_one(features)
    features_scaled = scaler_mejorado.transform_one(features)
    
    # Score y entrenamiento
    score = modelo_mejorado.score_one(features_scaled)
    scores_mejorados.append(score)
    features_mejoradas.append(features)
    
    modelo_mejorado.learn_one(features_scaled)

print("✅ Entrenamiento completado")


🤖 Entrenando modelo con detección de extremos...
🔄 Procesando con features mejoradas...
   Procesado: 0/450900 registros
   Procesado: 2000/450900 registros
   Procesado: 4000/450900 registros
   Procesado: 6000/450900 registros
   Procesado: 8000/450900 registros
   Procesado: 10000/450900 registros
   Procesado: 12000/450900 registros
   Procesado: 14000/450900 registros
   Procesado: 16000/450900 registros
   Procesado: 18000/450900 registros
   Procesado: 20000/450900 registros
   Procesado: 22000/450900 registros
   Procesado: 24000/450900 registros
   Procesado: 26000/450900 registros
   Procesado: 28000/450900 registros
   Procesado: 30000/450900 registros
   Procesado: 32000/450900 registros
   Procesado: 34000/450900 registros
   Procesado: 36000/450900 registros
   Procesado: 38000/450900 registros
   Procesado: 40000/450900 registros
   Procesado: 42000/450900 registros
   Procesado: 44000/450900 registros
   Procesado: 46000/450900 registros
   Procesado: 48000/450900 regi

# 8. CALCULAR UMBRAL GLOBAL

In [8]:
# Análisis detallado de scores para encontrar umbral óptimo
scores_array = np.array(scores_mejorados)

# Calcular múltiples percentiles
percentiles = [90, 92, 94, 95, 96, 97, 98, 99, 99.5]
umbrales = {}

print(f"\n📊 ANÁLISIS DE UMBRALES POSIBLES:")
for p in percentiles:
    umbral = np.percentile(scores_array, p)
    tasa_anomalia = (scores_array > umbral).mean() * 100
    umbrales[p] = {'umbral': umbral, 'tasa': tasa_anomalia}
    print(f"P{p}: Umbral={umbral:.4f}, Tasa={tasa_anomalia:.2f}%")

# Seleccionar umbral basado en PERCENTIL_BASE configurado
umbral_base = umbrales[PERCENTIL_BASE]['umbral']
umbral_objetivo = umbral_base * AJUSTE_UMBRAL  # 🔧 Ajuste configurable

# Calcular tasa final
tasa_objetivo = (scores_array > umbral_objetivo).mean() * 100

print(f"\n🎯 UMBRAL SELECCIONADO:")
print(f"📊 Percentil base: P{PERCENTIL_BASE}")
print(f"🔢 Umbral final: {umbral_objetivo:.4f}")
print(f"📈 Tasa estimada: {tasa_objetivo:.2f}%")
print(f"⚙️ Ajuste aplicado: {AJUSTE_UMBRAL}")


📊 ANÁLISIS DE UMBRALES POSIBLES:
P90: Umbral=0.8191, Tasa=10.00%
P92: Umbral=0.8479, Tasa=8.00%
P94: Umbral=0.8840, Tasa=6.00%
P95: Umbral=0.8990, Tasa=5.00%
P96: Umbral=0.9115, Tasa=4.00%
P97: Umbral=0.9233, Tasa=3.00%
P98: Umbral=0.9417, Tasa=2.00%
P99: Umbral=0.9724, Tasa=1.00%
P99.5: Umbral=0.9967, Tasa=0.50%

🎯 UMBRAL SELECCIONADO:
📊 Percentil base: P99
🔢 Umbral final: 0.9724
📈 Tasa estimada: 1.00%
⚙️ Ajuste aplicado: 1.0


# 9. GUARDAR MODELO Y CONFIGURACIÓN


In [9]:
print(f"\n💾 Guardando modelo mejorado...")

# Guardar modelo
modelo_path = os.path.join(MODELS_PATH, "modelo_general.pkl")
with open(modelo_path, 'wb') as f:
    pickle.dump(modelo_mejorado, f)

# Guardar scaler
scaler_path = os.path.join(MODELS_PATH, "scaler_general.pkl")
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler_mejorado, f)

# Guardar configuración
config_general = {
    'umbral_global': umbral_objetivo,
    'stats_por_pais': stats_dict,
    'fecha_entrenamiento': datetime.now().isoformat(),
    'n_trees': N_TREES,
    'tree_height': TREE_HEIGHT,
    'registros_entrenamiento': len(df_warmup),
    'paises_entrenamiento': df_warmup['CODIGODEPAIS'].nunique(),
    'parametros_features': {
        'peso_minutos_normal': PESO_MINUTOS_NORMAL,
        'peso_minutos_extremos': PESO_MINUTOS_EXTREMOS,
        'umbral_minutos_extremos': UMBRAL_MINUTOS_EXTREMOS,
        'peso_destinos': PESO_DESTINOS,
        'peso_spray_ratio': PESO_SPRAY_RATIO
    },
    'parametros_umbral': {
        'percentil_base': PERCENTIL_BASE,
        'ajuste_umbral': AJUSTE_UMBRAL
    }
}

config_path = os.path.join(MODELS_PATH, "config_modelo_general.pkl")
with open(config_path, 'wb') as f:
    pickle.dump(config_general, f)

print("✅ Modelo mejorado guardado exitosamente")


💾 Guardando modelo mejorado...
✅ Modelo mejorado guardado exitosamente


# 10. FUNCIÓN DE PREDICCIÓN GENERAL


In [10]:
def predecir_anomalia_mejorada(pais, linea, llamadas, minutos, destinos, modelo, scaler, umbral, stats_dict):
    """
    Predicción con detección inteligente de minutos extremos y spray calling
    """
    # Crear row simulado
    row_data = {
        'CODIGODEPAIS': pais,
        'N_LLAMADAS': llamadas,
        'N_MINUTOS': minutos,
        'N_DESTINOS': destinos
    }
    
    # Crear features mejoradas
    features = crear_features_contextualizadas_mejorada(row_data, stats_dict)
    
    # Normalizar
    features_scaled = scaler.transform_one(features)
    
    # Obtener score
    score = modelo.score_one(features_scaled)
    
    # LÓGICA MEJORADA PARA CONFIRMACIÓN DE ANOMALÍAS
    es_anomalia_base = score > umbral
    
    if es_anomalia_base:
        # Confirmar diferentes tipos de anomalías
        
        # Tipo 1: Minutos extremos (NUEVA DETECCIÓN)
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            es_anomalia_final = True
            razon = f"Minutos extremos ({minutos:.1f} min)"
        
        # Tipo 2: Spray calling confirmado
        elif destinos >= 6 and llamadas >= 12:
            es_anomalia_final = True
            razon = "Patrón de spray calling confirmado"
        
        # Tipo 3: Volumen excepcionalmente alto
        elif llamadas > 50 or destinos > 15:
            es_anomalia_final = True
            razon = "Volumen excepcionalmente alto"
        
        # Tipo 4: País de bajo tráfico con actividad sospechosa
        elif pais not in stats_dict or stats_dict.get(pais, {}).get('CATEGORIA') in ['Muy_Bajo', 'Bajo']:
            if destinos >= 4 and llamadas >= 8:
                es_anomalia_final = True
                razon = "Actividad sospechosa en país de bajo tráfico"
            else:
                es_anomalia_final = False
                razon = "Actividad baja en país de bajo tráfico"
        
        # Reglas de exclusión
        elif destinos < 3:
            es_anomalia_final = False
            razon = "Muy pocos destinos (<3)"
        elif destinos / max(llamadas, 1) < 0.15:
            es_anomalia_final = False
            razon = "Ratio destinos/llamadas muy bajo"
        elif llamadas < 5:
            es_anomalia_final = False
            razon = "Muy pocas llamadas (<5)"
        else:
            es_anomalia_final = False
            razon = "No cumple criterios de confirmación"
    else:
        es_anomalia_final = False
        razon = "Score bajo umbral"
    
    # Determinar contexto
    if pais in stats_dict:
        categoria = stats_dict[pais]['CATEGORIA']
        tipo_contexto = categoria
    else:
        tipo_contexto = "Muy_Bajo"  # 🔧 Países nuevos siempre como Muy_Bajo
    
    return {
        'score': score,
        'umbral': umbral,
        'es_anomalia': es_anomalia_final,
        'tipo_contexto': tipo_contexto,
        'razon_decision': razon,
        'features': features
    }

print("🔮 Función de predicción mejorada definida")

🔮 Función de predicción mejorada definida


# 11. SCORING EN PERÍODO DE PRUEBA


In [11]:
print(f"\n🎯 Scoring con modelo mejorado...")

resultados_mejorados = []

for contador, (idx, row) in enumerate(df_scoring.iterrows()):
    if contador % 2000 == 0:
        print(f"   Scoring: {contador}/{len(df_scoring)} registros")
    
    resultado = predecir_anomalia_mejorada(
        pais=row['CODIGODEPAIS'],
        linea=row['LINEA'],
        llamadas=row['N_LLAMADAS'],
        minutos=row['N_MINUTOS'],
        destinos=row['N_DESTINOS'],
        modelo=modelo_mejorado,
        scaler=scaler_mejorado,
        umbral=umbral_objetivo,
        stats_dict=stats_dict
    )
    
    resultado_completo = {
        'FECHA': row['FECHA'],
        'CODIGODEPAIS': row['CODIGODEPAIS'],
        'LINEA': row['LINEA'],
        'N_LLAMADAS': row['N_LLAMADAS'],
        'N_MINUTOS': row['N_MINUTOS'],
        'N_DESTINOS': row['N_DESTINOS'],
        'score_anomalia': resultado['score'],
        'umbral': resultado['umbral'],
        'es_anomalia': resultado['es_anomalia'],
        'tipo_contexto': resultado['tipo_contexto'],
        'razon_decision': resultado['razon_decision']
    }
    
    resultados_mejorados.append(resultado_completo)

df_resultados_mejorados = pd.DataFrame(resultados_mejorados)

print("✅ Scoring mejorado completado")


🎯 Scoring con modelo mejorado...
   Scoring: 0/343910 registros
   Scoring: 2000/343910 registros
   Scoring: 4000/343910 registros
   Scoring: 6000/343910 registros
   Scoring: 8000/343910 registros
   Scoring: 10000/343910 registros
   Scoring: 12000/343910 registros
   Scoring: 14000/343910 registros
   Scoring: 16000/343910 registros
   Scoring: 18000/343910 registros
   Scoring: 20000/343910 registros
   Scoring: 22000/343910 registros
   Scoring: 24000/343910 registros
   Scoring: 26000/343910 registros
   Scoring: 28000/343910 registros
   Scoring: 30000/343910 registros
   Scoring: 32000/343910 registros
   Scoring: 34000/343910 registros
   Scoring: 36000/343910 registros
   Scoring: 38000/343910 registros
   Scoring: 40000/343910 registros
   Scoring: 42000/343910 registros
   Scoring: 44000/343910 registros
   Scoring: 46000/343910 registros
   Scoring: 48000/343910 registros
   Scoring: 50000/343910 registros
   Scoring: 52000/343910 registros
   Scoring: 54000/343910 regi

# 12. ANÁLISIS DETALLADO DE RESULTADOS


In [12]:
print(f"\n📊 RESULTADOS CON MODELO MEJORADO:")
print(f"📞 Total de líneas evaluadas: {len(df_resultados_mejorados)}")
print(f"🚨 Anomalías detectadas: {df_resultados_mejorados['es_anomalia'].sum()}")
print(f"📈 Tasa de anomalías: {df_resultados_mejorados['es_anomalia'].mean()*100:.3f}%")

# Análisis de razones de decisión
print(f"\n🔍 RAZONES DE DECISIONES:")
razones = df_resultados_mejorados['razon_decision'].value_counts()
print(razones)

# Anomalías confirmadas
anomalias_confirmadas = df_resultados_mejorados[df_resultados_mejorados['es_anomalia'] == True]
if len(anomalias_confirmadas) > 0:
    print(f"\n🎯 ANOMALÍAS CONFIRMADAS (Top 10):")
    print(anomalias_confirmadas.sort_values('score_anomalia', ascending=False)[
        ['CODIGODEPAIS', 'LINEA', 'N_LLAMADAS', 'N_MINUTOS', 'N_DESTINOS', 
         'score_anomalia', 'razon_decision']
    ].head(10))
    
    # Estadísticas de anomalías confirmadas
    print(f"\n📊 ESTADÍSTICAS DE ANOMALÍAS CONFIRMADAS:")
    print(f"📞 Llamadas - Min: {anomalias_confirmadas['N_LLAMADAS'].min()}, Max: {anomalias_confirmadas['N_LLAMADAS'].max()}, Media: {anomalias_confirmadas['N_LLAMADAS'].mean():.1f}")
    print(f"🎯 Destinos - Min: {anomalias_confirmadas['N_DESTINOS'].min()}, Max: {anomalias_confirmadas['N_DESTINOS'].max()}, Media: {anomalias_confirmadas['N_DESTINOS'].mean():.1f}")
    print(f"⏱️ Minutos - Min: {anomalias_confirmadas['N_MINUTOS'].min()}, Max: {anomalias_confirmadas['N_MINUTOS'].max()}, Media: {anomalias_confirmadas['N_MINUTOS'].mean():.1f}")
    print(f"📊 Ratio Destinos/Llamadas promedio: {(anomalias_confirmadas['N_DESTINOS']/anomalias_confirmadas['N_LLAMADAS']).mean():.3f}")
    
    # Análisis de minutos extremos
    minutos_extremos = anomalias_confirmadas[anomalias_confirmadas['razon_decision'].str.contains('Minutos extremos')]
    if len(minutos_extremos) > 0:
        print(f"\n⚡ DETECCIONES POR MINUTOS EXTREMOS: {len(minutos_extremos)}")
        print(f"⏱️ Minutos promedio en extremos: {minutos_extremos['N_MINUTOS'].mean():.1f}")
else:
    print("ℹ️ No se detectaron anomalías")

print(f"\n✅ MEJORAS IMPLEMENTADAS:")
print(f"🔧 Detección inteligente de minutos extremos (≥{UMBRAL_MINUTOS_EXTREMOS} min)")
print(f"🌍 Países nuevos clasificados como 'Muy_Bajo' automáticamente")
print(f"📊 Umbral configurable (P{PERCENTIL_BASE} × {AJUSTE_UMBRAL})")
print(f"🛡️ Reglas mejoradas anti-falsos positivos")
print(f"🎯 Mayor sensibilidad para países de bajo tráfico")


📊 RESULTADOS CON MODELO MEJORADO:
📞 Total de líneas evaluadas: 343910
🚨 Anomalías detectadas: 2507
📈 Tasa de anomalías: 0.729%

🔍 RAZONES DE DECISIONES:
razon_decision
Score bajo umbral                         341175
Patrón de spray calling confirmado          2378
No cumple criterios de confirmación          127
Actividad baja en país de bajo tráfico        98
Ratio destinos/llamadas muy bajo               3
                                           ...  
Minutos extremos (597.6 min)                   1
Minutos extremos (558.3 min)                   1
Minutos extremos (518.3 min)                   1
Minutos extremos (419.6 min)                   1
Minutos extremos (350.1 min)                   1
Name: count, Length: 129, dtype: int64

🎯 ANOMALÍAS CONFIRMADAS (Top 10):
        CODIGODEPAIS        LINEA  N_LLAMADAS  N_MINUTOS  N_DESTINOS  \
70256            593  51880609481         327     726.00         308   
76478            593  51880609481         283     829.47         272   
24

# 13. GUARDAR RESULTADOS FINALES


In [13]:
# Guardar todos los resultados
resultados_path = os.path.join(MODELS_PATH, "resultados_modelo_general.csv")
df_resultados_mejorados.to_csv(resultados_path, index=False)

# Guardar solo anomalías si existen
if len(anomalias_confirmadas) > 0:
    anomalias_path = os.path.join(MODELS_PATH, "anomalias_modelo_general.csv")
    anomalias_confirmadas.to_csv(anomalias_path, index=False)
    print(f"🚨 Solo anomalías: {anomalias_path}")
else:
    print("ℹ️ No hay anomalías para guardar por separado")

print(f"\n💾 RESULTADOS GUARDADOS:")
print(f"📄 Todos los resultados: {resultados_path}")

🚨 Solo anomalías: C:\Users\User\Desktop\TESIS\Modelos\anomalias_modelo_general.csv

💾 RESULTADOS GUARDADOS:
📄 Todos los resultados: C:\Users\User\Desktop\TESIS\Modelos\resultados_modelo_general.csv


# # 14. FUNCIÓN PARA PROBAR REGISTROS INDIVIDUALES


In [14]:
def probar_registro_individual(pais, linea, llamadas, minutos, destinos, mostrar_detalles=True):
    """
    Función para probar un registro individual y ver el análisis completo
    
    Parámetros:
    - pais: Código del país (ej: "1", "34", "591")
    - linea: Número de línea telefónica (ej: "70123456")
    - llamadas: Número de llamadas realizadas
    - minutos: Minutos totales de las llamadas
    - destinos: Número de destinos únicos
    - mostrar_detalles: Si mostrar análisis detallado (True/False)
    """
    print(f"\n🔍 ANÁLISIS DE REGISTRO INDIVIDUAL")
    print(f"=" * 50)
    print(f"🌍 País: {pais}")
    print(f"📞 Línea: {linea}")
    print(f"📊 Llamadas: {llamadas}")
    print(f"⏱️ Minutos: {minutos}")
    print(f"🎯 Destinos: {destinos}")
    print(f"-" * 50)
    
    # Realizar predicción
    resultado = predecir_anomalia_mejorada(
        pais=pais,
        linea=linea,
        llamadas=llamadas,
        minutos=minutos,
        destinos=destinos,
        modelo=modelo_mejorado,
        scaler=scaler_mejorado,
        umbral=umbral_objetivo,
        stats_dict=stats_dict
    )
    
    # Mostrar resultado principal
    if resultado['es_anomalia']:
        print(f"🚨 RESULTADO: ANOMALÍA DETECTADA")
        print(f"🔴 Score: {resultado['score']:.4f} (Umbral: {resultado['umbral']:.4f})")
    else:
        print(f"✅ RESULTADO: REGISTRO NORMAL")
        print(f"🟢 Score: {resultado['score']:.4f} (Umbral: {resultado['umbral']:.4f})")
    
    print(f"🏷️ Contexto del país: {resultado['tipo_contexto']}")
    print(f"💭 Razón: {resultado['razon_decision']}")
    
    if mostrar_detalles:
        print(f"\n📋 ANÁLISIS DETALLADO:")
        print(f"-" * 30)
        
        # Mostrar contexto del país
        if pais in stats_dict:
            pais_stats = stats_dict[pais]
            print(f"📊 Estadísticas del país {pais}:")
            print(f"   📞 Llamadas P95: {pais_stats['LLAMADAS_P95']:.1f}")
            print(f"   ⏱️ Minutos P95: {pais_stats['MINUTOS_P95']:.1f}")
            print(f"   🎯 Destinos P95: {pais_stats['DESTINOS_P95']:.1f}")
            print(f"   📈 Categoría: {pais_stats['CATEGORIA']}")
        else:
            print(f"⚠️ País {pais} no encontrado en estadísticas (país nuevo)")
        
        # Mostrar features calculadas
        print(f"\n🔧 CARACTERÍSTICAS CALCULADAS:")
        features = resultado['features']
        for key, value in features.items():
            if value > 0:  # Solo mostrar features activas
                print(f"   {key}: {value:.3f}")
        
        # Análisis de patrones
        print(f"\n🎯 ANÁLISIS DE PATRONES:")
        ratio_destinos = destinos / max(llamadas, 1)
        minutos_por_llamada = minutos / max(llamadas, 1)
        
        print(f"   📊 Ratio destinos/llamadas: {ratio_destinos:.3f}")
        print(f"   ⏱️ Minutos por llamada: {minutos_por_llamada:.2f}")
        
        # Indicadores de riesgo
        print(f"\n⚠️ INDICADORES DE RIESGO:")
        
        if destinos >= 10 and llamadas >= 20:
            print(f"   🔴 Patrón de spray calling fuerte detectado")
        elif destinos >= 6 and llamadas >= 12:
            print(f"   🟡 Patrón de spray calling medio detectado")
        
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            print(f"   🔴 Minutos extremos detectados (≥{UMBRAL_MINUTOS_EXTREMOS})")
        
        if ratio_destinos >= 0.5:
            print(f"   🟡 Alta diversidad de destinos (≥50%)")
        
        if minutos_por_llamada < 2:
            print(f"   🟡 Llamadas muy cortas (posible exploración)")
        
        if llamadas > 50 or destinos > 15:
            print(f"   🔴 Volumen excepcionalmente alto")
    
    print(f"=" * 50)
    return resultado

In [37]:
probar_registro_individual(pais=1, linea='70123456', llamadas=12, minutos=20, destinos=1)



🔍 ANÁLISIS DE REGISTRO INDIVIDUAL
🌍 País: 1
📞 Línea: 70123456
📊 Llamadas: 12
⏱️ Minutos: 20
🎯 Destinos: 1
--------------------------------------------------
✅ RESULTADO: REGISTRO NORMAL
🟢 Score: 0.9002 (Umbral: 0.9724)
🏷️ Contexto del país: Alto
💭 Razón: Score bajo umbral

📋 ANÁLISIS DETALLADO:
------------------------------
📊 Estadísticas del país 1:
   📞 Llamadas P95: 5.0
   ⏱️ Minutos P95: 49.4
   🎯 Destinos P95: 2.0
   📈 Categoría: Alto

🔧 CARACTERÍSTICAS CALCULADAS:
   llamadas_norm: 1.200
   destinos_norm: 0.600
   minutos_norm: 0.349
   diversidad_destinos: 0.083
   llamadas_por_destino: 1.000
   eficiencia_destinos: 0.167
   factor_pais_bajo: 1.000
   factor_pais_alto: 0.900

🎯 ANÁLISIS DE PATRONES:
   📊 Ratio destinos/llamadas: 0.083
   ⏱️ Minutos por llamada: 1.67

⚠️ INDICADORES DE RIESGO:
   🟡 Llamadas muy cortas (posible exploración)


{'score': 0.9002249633610161,
 'umbral': np.float64(0.9724393551538845),
 'es_anomalia': False,
 'tipo_contexto': 'Alto',
 'razon_decision': 'Score bajo umbral',
 'features': {'llamadas_norm': 1.2000000000000002,
  'destinos_norm': 0.6,
  'minutos_norm': np.float64(0.3488690925491036),
  'diversidad_destinos': 0.08333333333333333,
  'spray_ratio': 0,
  'minutos_extremos': 0.0,
  'minutos_sospechosos': 0.0,
  'patron_spray_fuerte': 0.0,
  'patron_spray_medio': 0.0,
  'alta_diversidad': 0,
  'volumen_llamadas_alto': 0,
  'volumen_destinos_alto': 0,
  'llamadas_por_destino': 1,
  'eficiencia_destinos': 0.16666666666666666,
  'factor_pais_bajo': 1.0,
  'factor_pais_alto': 0.9}}

# # 15. REINICIALIZACIÓN Y CARGA DEL MODELO ENTRENADO

In [2]:
import pandas as pd
import numpy as np
from river import anomaly
from river import preprocessing
import pickle
import os
from datetime import datetime
import warnings
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


# # 16. CONFIGURACIÓN DE RUTAS Y CARGA DE MODELO

In [3]:
# Rutas de archivos
MODELS_PATH = r"C:\Users\User\Desktop\TESIS\Modelos"
EVALUATION_CSV_PATH = r"C:\Users\User\Desktop\TESIS\NuevoDataSet\DataSetFinalProbarMatriz.csv"  # 🔧 CAMBIAR ESTA RUTA

# Verificar que el directorio de modelos existe
if not os.path.exists(MODELS_PATH):
    print(f"❌ Error: Directorio de modelos no encontrado: {MODELS_PATH}")
    exit()

print(f"📁 Directorio de modelos: {MODELS_PATH}")

📁 Directorio de modelos: C:\Users\User\Desktop\TESIS\Modelos


# # 17. CARGAR MODELO, SCALER Y CONFIGURACIÓN

In [4]:
print("\n🔄 Cargando modelo entrenado...")

# Cargar modelo
modelo_path = os.path.join(MODELS_PATH, "modelo_general.pkl")
with open(modelo_path, 'rb') as f:
    modelo_cargado = pickle.load(f)

# Cargar scaler
scaler_path = os.path.join(MODELS_PATH, "scaler_general.pkl")
with open(scaler_path, 'rb') as f:
    scaler_cargado = pickle.load(f)

# Cargar configuración
config_path = os.path.join(MODELS_PATH, "config_modelo_general.pkl")
with open(config_path, 'rb') as f:
    config_cargado = pickle.load(f)

print("✅ Modelo cargado exitosamente")

# Mostrar información del modelo
print(f"\n📊 INFORMACIÓN DEL MODELO CARGADO:")
print(f"🎯 Umbral global: {config_cargado['umbral_global']:.4f}")
print(f"🌍 Países en entrenamiento: {config_cargado['paises_entrenamiento']}")
print(f"📈 Registros de entrenamiento: {config_cargado['registros_entrenamiento']}")
print(f"📅 Fecha de entrenamiento: {config_cargado['fecha_entrenamiento']}")
print(f"🌳 Número de árboles: {config_cargado['n_trees']}")
print(f"📏 Altura de árboles: {config_cargado['tree_height']}")

# Extraer configuraciones
umbral_global = config_cargado['umbral_global']
stats_por_pais_dict = config_cargado['stats_por_pais']
parametros_features = config_cargado['parametros_features']

# Configurar parámetros de features
PESO_MINUTOS_NORMAL = parametros_features['peso_minutos_normal']
PESO_MINUTOS_EXTREMOS = parametros_features['peso_minutos_extremos']
UMBRAL_MINUTOS_EXTREMOS = parametros_features['umbral_minutos_extremos']
PESO_DESTINOS = parametros_features['peso_destinos']
PESO_SPRAY_RATIO = parametros_features['peso_spray_ratio']

print(f"\n⚙️ PARÁMETROS DE FEATURES CARGADOS:")
print(f"🔧 Peso minutos normal: {PESO_MINUTOS_NORMAL}")
print(f"🔧 Peso minutos extremos: {PESO_MINUTOS_EXTREMOS}")
print(f"🔧 Umbral minutos extremos: {UMBRAL_MINUTOS_EXTREMOS}")
print(f"🔧 Peso destinos: {PESO_DESTINOS}")
print(f"🔧 Peso spray ratio: {PESO_SPRAY_RATIO}")


🔄 Cargando modelo entrenado...
✅ Modelo cargado exitosamente

📊 INFORMACIÓN DEL MODELO CARGADO:
🎯 Umbral global: 0.9724
🌍 Países en entrenamiento: 188
📈 Registros de entrenamiento: 450900
📅 Fecha de entrenamiento: 2025-06-09T18:10:07.358019
🌳 Número de árboles: 80
📏 Altura de árboles: 10

⚙️ PARÁMETROS DE FEATURES CARGADOS:
🔧 Peso minutos normal: 0.4
🔧 Peso minutos extremos: 1.5
🔧 Umbral minutos extremos: 300
🔧 Peso destinos: 1.2
🔧 Peso spray ratio: 1.5


# # 18. REDEFINIR FUNCIONES NECESARIAS

In [5]:
def crear_features_contextualizadas_mejorada(row, stats_pais_dict):
    """
    Crea características balanceadas que detecten minutos extremos y spray calling
    """
    pais = row['CODIGODEPAIS']
    llamadas = row['N_LLAMADAS']
    minutos = row['N_MINUTOS']
    destinos = row['N_DESTINOS']
    
    # Obtener contexto del país (si existe)
    if pais in stats_pais_dict:
        pais_stats = stats_pais_dict[pais]
        categoria = pais_stats['CATEGORIA']
        
        # Normalizar por el contexto del país
        llamadas_norm = min(llamadas / max(pais_stats['LLAMADAS_P95'], 1), 1.5)
        destinos_norm = min(destinos / max(pais_stats['DESTINOS_P95'], 1), 1.5)
        
        # MEJORA: Detección inteligente de minutos extremos
        minutos_p90 = pais_stats.get('MINUTOS_P90', pais_stats['MINUTOS_P95'] * 0.9)
        minutos_p95 = pais_stats['MINUTOS_P95']
        
        # Transformación adaptativa de minutos
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:  # 🔧 Minutos extremos
            minutos_norm = min(minutos / max(minutos_p90, 1), 3.0)  # Mayor rango para extremos
            peso_minutos = PESO_MINUTOS_EXTREMOS  # Peso alto para extremos
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(max(minutos_p90, 1)), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL  # Peso normal
            
    else:
        # País nuevo - SIEMPRE clasificar como 'Muy_Bajo'
        categoria = 'Muy_Bajo'
        llamadas_norm = min(llamadas / 10, 2.0)  # Más sensible para países nuevos
        destinos_norm = min(destinos / 5, 2.0)   # Más sensible para países nuevos
        
        # Para países nuevos, ser más sensible a minutos altos
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / 50, 3.0)  # Muy sensible a minutos extremos
            peso_minutos = PESO_MINUTOS_EXTREMOS * 1.2  # Peso extra para países nuevos
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(60), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
    
    # Features principales - REBALANCEADAS con detección de extremos
    features = {
        # 1. Valores normalizados con peso adaptativo
        'llamadas_norm': llamadas_norm * 0.8,
        'destinos_norm': destinos_norm * PESO_DESTINOS,  # 🔧 Ajustable
        'minutos_norm': minutos_norm * peso_minutos,     # 🔧 Peso adaptativo
        
        # 2. Ratios críticos para fraude
        'diversidad_destinos': min(destinos / max(llamadas, 1), 1.0),
        'spray_ratio': min(destinos / max(llamadas, 1) * PESO_SPRAY_RATIO, 1.0) if destinos >= 5 else 0,
        
        # 3. NUEVA: Detección específica de minutos extremos
        'minutos_extremos': 1.0 if minutos >= UMBRAL_MINUTOS_EXTREMOS else 0.0,
        'minutos_sospechosos': min((minutos - 200) / 300, 1.0) if minutos > 200 else 0.0,
        
        # 4. Patrones de fraude
        'patron_spray_fuerte': 1.0 if (destinos >= 10 and llamadas >= 20) else 0.0,
        'patron_spray_medio': 0.5 if (destinos >= 6 and llamadas >= 12) else 0.0,
        'alta_diversidad': min(destinos / 12, 1) if destinos >= 5 else 0,
        
        # 5. Indicadores de volumen anómalo
        'volumen_llamadas_alto': min((llamadas - 30) / 50, 1) if llamadas > 30 else 0,
        'volumen_destinos_alto': min((destinos - 10) / 20, 1) if destinos > 10 else 0,
        
        # 6. Características de comportamiento
        'llamadas_por_destino': min(llamadas / max(destinos, 1) / 5, 1),
        'eficiencia_destinos': min(destinos / max(llamadas * 0.5, 1), 1),
        
        # 7. MEJORA: Ajuste por contexto de país
        'factor_pais_bajo': 1.5 if categoria in ['Muy_Bajo', 'Bajo'] else 1.0,  # Más sensible
        'factor_pais_alto': 0.9 if categoria in ['Alto', 'Medio'] else 1.0      # Menos sensible
    }
    
    return features

def predecir_anomalia_mejorada(pais, linea, llamadas, minutos, destinos, modelo, scaler, umbral, stats_dict):
    """
    Predicción con detección inteligente de minutos extremos y spray calling
    """
    # Crear row simulado
    row_data = {
        'CODIGODEPAIS': pais,
        'N_LLAMADAS': llamadas,
        'N_MINUTOS': minutos,
        'N_DESTINOS': destinos
    }
    
    # Crear features mejoradas
    features = crear_features_contextualizadas_mejorada(row_data, stats_dict)
    
    # Normalizar
    features_scaled = scaler.transform_one(features)
    
    # Obtener score
    score = modelo.score_one(features_scaled)
    
    # LÓGICA MEJORADA PARA CONFIRMACIÓN DE ANOMALÍAS
    es_anomalia_base = score > umbral
    
    if es_anomalia_base:
        # Confirmar diferentes tipos de anomalías
        
        # Tipo 1: Minutos extremos (NUEVA DETECCIÓN)
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            es_anomalia_final = True
            razon = f"Minutos extremos ({minutos:.1f} min)"
        
        # Tipo 2: Spray calling confirmado
        elif destinos >= 6 and llamadas >= 12:
            es_anomalia_final = True
            razon = "Patrón de spray calling confirmado"
        
        # Tipo 3: Volumen excepcionalmente alto
        elif llamadas > 50 or destinos > 15:
            es_anomalia_final = True
            razon = "Volumen excepcionalmente alto"
        
        # Tipo 4: País de bajo tráfico con actividad sospechosa
        elif pais not in stats_dict or stats_dict.get(pais, {}).get('CATEGORIA') in ['Muy_Bajo', 'Bajo']:
            if destinos >= 4 and llamadas >= 8:
                es_anomalia_final = True
                razon = "Actividad sospechosa en país de bajo tráfico"
            else:
                es_anomalia_final = False
                razon = "Actividad baja en país de bajo tráfico"
        
        # Reglas de exclusión
        elif destinos < 3:
            es_anomalia_final = False
            razon = "Muy pocos destinos (<3)"
        elif destinos / max(llamadas, 1) < 0.15:
            es_anomalia_final = False
            razon = "Ratio destinos/llamadas muy bajo"
        elif llamadas < 5:
            es_anomalia_final = False
            razon = "Muy pocas llamadas (<5)"
        else:
            es_anomalia_final = False
            razon = "No cumple criterios de confirmación"
    else:
        es_anomalia_final = False
        razon = "Score bajo umbral"
    
    # Determinar contexto
    if pais in stats_dict:
        categoria = stats_dict[pais]['CATEGORIA']
        tipo_contexto = categoria
    else:
        tipo_contexto = "Muy_Bajo"  # 🔧 Países nuevos siempre como Muy_Bajo
    
    return {
        'score': score,
        'umbral': umbral,
        'es_anomalia': es_anomalia_final,
        'tipo_contexto': tipo_contexto,
        'razon_decision': razon,
        'features': features
    }

print("🔧 Funciones redefinidas correctamente")

🔧 Funciones redefinidas correctamente


# # 19. CARGAR DATASET DE EVALUACIÓN CON ETIQUETAS DE FRAUDE


In [6]:
print(f"\n📂 Cargando dataset de evaluación...")

# Verificar que el archivo existe
if not os.path.exists(EVALUATION_CSV_PATH):
    print(f"❌ Error: Archivo de evaluación no encontrado: {EVALUATION_CSV_PATH}")
    print(f"📝 Por favor, asegúrate de que el archivo exista y contenga las columnas:")
    print(f"   - FECHA, CODIGODEPAIS, LINEA, N_LLAMADAS, N_MINUTOS, N_DESTINOS, FRAUDE")
    print(f"   - FRAUDE debe ser 1 (fraudulento) o 0 (normal)")
    exit()

# Cargar dataset
df_evaluacion = pd.read_csv(EVALUATION_CSV_PATH)

# Convertir fecha a datetime si existe
if 'FECHA' in df_evaluacion.columns:
    df_evaluacion['FECHA'] = pd.to_datetime(df_evaluacion['FECHA'], format='%d/%m/%Y', errors='coerce')

print(f"✅ Dataset cargado - Shape: {df_evaluacion.shape}")

# Verificar columnas requeridas
columnas_requeridas = ['CODIGODEPAIS', 'LINEA', 'N_LLAMADAS', 'N_MINUTOS', 'N_DESTINOS', 'FRAUDE']
columnas_faltantes = [col for col in columnas_requeridas if col not in df_evaluacion.columns]

if columnas_faltantes:
    print(f"❌ Error: Columnas faltantes: {columnas_faltantes}")
    print(f"📋 Columnas disponibles: {list(df_evaluacion.columns)}")
    exit()

# Verificar valores de FRAUDE
valores_fraude = df_evaluacion['FRAUDE'].unique()
if not all(v in [0, 1] for v in valores_fraude):
    print(f"❌ Error: FRAUDE debe contener solo valores 0 o 1. Valores encontrados: {valores_fraude}")
    exit()

print(f"🔍 ANÁLISIS DEL DATASET DE EVALUACIÓN:")
print(f"📊 Total de registros: {len(df_evaluacion)}")
print(f"🚨 Casos de fraude: {df_evaluacion['FRAUDE'].sum()} ({df_evaluacion['FRAUDE'].mean()*100:.2f}%)")
print(f"✅ Casos normales: {(df_evaluacion['FRAUDE'] == 0).sum()} ({(df_evaluacion['FRAUDE'] == 0).mean()*100:.2f}%)")
print(f"🌍 Países únicos: {df_evaluacion['CODIGODEPAIS'].nunique()}")
print(f"📞 Líneas únicas: {df_evaluacion['LINEA'].nunique()}")

# Mostrar estadísticas por clase
print(f"\n📊 ESTADÍSTICAS POR CLASE:")
print(f"CASOS NORMALES (FRAUDE = 0):")
normales = df_evaluacion[df_evaluacion['FRAUDE'] == 0]
print(f"  📞 Llamadas - Min: {normales['N_LLAMADAS'].min()}, Max: {normales['N_LLAMADAS'].max()}, Media: {normales['N_LLAMADAS'].mean():.1f}")
print(f"  ⏱️ Minutos - Min: {normales['N_MINUTOS'].min()}, Max: {normales['N_MINUTOS'].max()}, Media: {normales['N_MINUTOS'].mean():.1f}")
print(f"  🎯 Destinos - Min: {normales['N_DESTINOS'].min()}, Max: {normales['N_DESTINOS'].max()}, Media: {normales['N_DESTINOS'].mean():.1f}")

if df_evaluacion['FRAUDE'].sum() > 0:
    print(f"\nCASOS DE FRAUDE (FRAUDE = 1):")
    fraudes = df_evaluacion[df_evaluacion['FRAUDE'] == 1]
    print(f"  📞 Llamadas - Min: {fraudes['N_LLAMADAS'].min()}, Max: {fraudes['N_LLAMADAS'].max()}, Media: {fraudes['N_LLAMADAS'].mean():.1f}")
    print(f"  ⏱️ Minutos - Min: {fraudes['N_MINUTOS'].min()}, Max: {fraudes['N_MINUTOS'].max()}, Media: {fraudes['N_MINUTOS'].mean():.1f}")
    print(f"  🎯 Destinos - Min: {fraudes['N_DESTINOS'].min()}, Max: {fraudes['N_DESTINOS'].max()}, Media: {fraudes['N_DESTINOS'].mean():.1f}")



📂 Cargando dataset de evaluación...
✅ Dataset cargado - Shape: (102684, 7)
🔍 ANÁLISIS DEL DATASET DE EVALUACIÓN:
📊 Total de registros: 102684
🚨 Casos de fraude: 2813 (2.74%)
✅ Casos normales: 99871 (97.26%)
🌍 Países únicos: 183
📞 Líneas únicas: 71752

📊 ESTADÍSTICAS POR CLASE:
CASOS NORMALES (FRAUDE = 0):
  📞 Llamadas - Min: 0, Max: 200, Media: 1.5
  ⏱️ Minutos - Min: 0.02, Max: 854.79, Media: 3.5
  🎯 Destinos - Min: 0, Max: 121, Media: 1.2

CASOS DE FRAUDE (FRAUDE = 1):
  📞 Llamadas - Min: 1, Max: 295, Media: 57.3
  ⏱️ Minutos - Min: 0.02, Max: 459.02, Media: 82.7
  🎯 Destinos - Min: 1, Max: 293, Media: 55.5


# # 20. REALIZAR PREDICCIONES EN DATASET DE EVALUACIÓN


In [7]:
print(f"\n🔮 Realizando predicciones en dataset de evaluación...")

predicciones = []
scores = []

for contador, (idx, row) in enumerate(df_evaluacion.iterrows()):
    if contador % 1000 == 0:
        print(f"   Procesando: {contador}/{len(df_evaluacion)} registros")
    
    # Realizar predicción
    resultado = predecir_anomalia_mejorada(
        pais=row['CODIGODEPAIS'],
        linea=row['LINEA'],
        llamadas=row['N_LLAMADAS'],
        minutos=row['N_MINUTOS'],
        destinos=row['N_DESTINOS'],
        modelo=modelo_cargado,
        scaler=scaler_cargado,
        umbral=umbral_global,
        stats_dict=stats_por_pais_dict
    )
    
    # Guardar predicción (1 si es anomalía, 0 si es normal)
    predicciones.append(1 if resultado['es_anomalia'] else 0)
    scores.append(resultado['score'])

# Agregar predicciones al dataframe
df_evaluacion['PREDICCION'] = predicciones
df_evaluacion['SCORE_ANOMALIA'] = scores

print(f"✅ Predicciones completadas")


🔮 Realizando predicciones en dataset de evaluación...
   Procesando: 0/102684 registros
   Procesando: 1000/102684 registros
   Procesando: 2000/102684 registros
   Procesando: 3000/102684 registros
   Procesando: 4000/102684 registros
   Procesando: 5000/102684 registros
   Procesando: 6000/102684 registros
   Procesando: 7000/102684 registros
   Procesando: 8000/102684 registros
   Procesando: 9000/102684 registros
   Procesando: 10000/102684 registros
   Procesando: 11000/102684 registros
   Procesando: 12000/102684 registros
   Procesando: 13000/102684 registros
   Procesando: 14000/102684 registros
   Procesando: 15000/102684 registros
   Procesando: 16000/102684 registros
   Procesando: 17000/102684 registros
   Procesando: 18000/102684 registros
   Procesando: 19000/102684 registros
   Procesando: 20000/102684 registros
   Procesando: 21000/102684 registros
   Procesando: 22000/102684 registros
   Procesando: 23000/102684 registros
   Procesando: 24000/102684 registros
   Proce

# # 21. CALCULAR MÉTRICAS Y MATRIZ DE CONFUSIÓN


In [8]:
print(f"\n📊 CALCULANDO MÉTRICAS DE EVALUACIÓN...")

# Extraer etiquetas reales y predicciones
y_true = df_evaluacion['FRAUDE'].values
y_pred = df_evaluacion['PREDICCION'].values

# Calcular métricas principales
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print(f"🎯 MÉTRICAS DE EVALUACIÓN:")
print(f"📈 Accuracy (Exactitud): {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"🎯 Precision (Precisión): {precision:.4f} ({precision*100:.2f}%)")
print(f"🔍 Recall (Sensibilidad): {recall:.4f} ({recall*100:.2f}%)")
print(f"⚖️ F1-Score: {f1:.4f} ({f1*100:.2f}%)")

# Calcular matriz de confusión
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n📋 MATRIZ DE CONFUSIÓN:")
print(f"┌─────────────────┬─────────────────┐")
print(f"│     REAL \\ PRED │   Normal   │ Anomalía │")
print(f"├─────────────────┼─────────────────┤")
print(f"│      Normal     │   {tn:6d}   │  {fp:6d}  │")
print(f"│     Fraude      │   {fn:6d}   │  {tp:6d}  │")
print(f"└─────────────────┴─────────────────┘")

print(f"\n🔢 INTERPRETACIÓN:")
print(f"✅ Verdaderos Negativos (TN): {tn} - Casos normales correctamente identificados")
print(f"❌ Falsos Positivos (FP): {fp} - Casos normales incorrectamente marcados como fraude")
print(f"❌ Falsos Negativos (FN): {fn} - Casos de fraude no detectados")
print(f"✅ Verdaderos Positivos (TP): {tp} - Casos de fraude correctamente detectados")

# Calcular tasas adicionales
if (tn + fp) > 0:
    especificidad = tn / (tn + fp)
    print(f"🛡️ Especificidad (Tasa de Verdaderos Negativos): {especificidad:.4f} ({especificidad*100:.2f}%)")

if (fp + tn) > 0:
    tasa_fp = fp / (fp + tn)
    print(f"⚠️ Tasa de Falsos Positivos: {tasa_fp:.4f} ({tasa_fp*100:.2f}%)")

if (fn + tp) > 0:
    tasa_fn = fn / (fn + tp)
    print(f"⚠️ Tasa de Falsos Negativos: {tasa_fn:.4f} ({tasa_fn*100:.2f}%)")


📊 CALCULANDO MÉTRICAS DE EVALUACIÓN...
🎯 MÉTRICAS DE EVALUACIÓN:
📈 Accuracy (Exactitud): 0.9911 (99.11%)
🎯 Precision (Precisión): 0.8988 (89.88%)
🔍 Recall (Sensibilidad): 0.7611 (76.11%)
⚖️ F1-Score: 0.8243 (82.43%)

📋 MATRIZ DE CONFUSIÓN:
┌─────────────────┬─────────────────┐
│     REAL \ PRED │   Normal   │ Anomalía │
├─────────────────┼─────────────────┤
│      Normal     │    99630   │     241  │
│     Fraude      │      672   │    2141  │
└─────────────────┴─────────────────┘

🔢 INTERPRETACIÓN:
✅ Verdaderos Negativos (TN): 99630 - Casos normales correctamente identificados
❌ Falsos Positivos (FP): 241 - Casos normales incorrectamente marcados como fraude
❌ Falsos Negativos (FN): 672 - Casos de fraude no detectados
✅ Verdaderos Positivos (TP): 2141 - Casos de fraude correctamente detectados
🛡️ Especificidad (Tasa de Verdaderos Negativos): 0.9976 (99.76%)
⚠️ Tasa de Falsos Positivos: 0.0024 (0.24%)
⚠️ Tasa de Falsos Negativos: 0.2389 (23.89%)


# # 22. Guardar Resultados

In [9]:
resultados_path = os.path.join(MODELS_PATH, "resultados_evaluacion_completa.csv")
df_evaluacion.to_csv(resultados_path, index=False)
print(f"📄 Resultados completos: {resultados_path}")



📄 Resultados completos: C:\Users\User\Desktop\TESIS\Modelos\resultados_evaluacion_completa.csv
